# Toy version

In [1]:
import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from gym import spaces
from gym_trading_env.environments import TradingEnv

from tqdm import trange

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv('data/INTC_1Min_2023-08-01_2024-01-31.csv')
df.set_index('timestamp', inplace=True)

In [4]:
class POMDPTEnv(TradingEnv):
    def __init__(self, df, window_size=60, initial_balance=100_000,transaction_cost=2.3e-5, slippage=0.2, eta=0.01, alpha=0, beta=0):
        super().__init__(df=df)
        
        self.df = df
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.slippage = slippage

        self.observation_space = spaces.Box(
            low=-np.inf, 
            high=np.inf, 
            shape=(4 + 4 * window_size,), # OHLCV + 2 indicators + account
            )
        self.action_space = spaces.Discrete(3) # buy or sell

        # Reward variables
        self.eta = eta
        self.alpha = alpha
        self.beta = beta

        # Initialize
        self.reset()
        
    def _compute_dual_thrust(self, k1=0.3, k2=0.3):
        df = self.df.iloc[:self.current_step]
        n = self.window_size

        hh = df['high'].rolling(n).max().iloc[-1]
        hc = df['close'].rolling(n).max().iloc[-1]
        lc = df['close'].rolling(n).min().iloc[-1]
        ll = df['low'].rolling(n).min().iloc[-1]

        self.range = max(hh - lc, hc - ll)
        self.buy_line = df['open'].iloc[-1] + k1 * self.range
        self.sell_line = df['open'].iloc[-1] - k2 * self.range

    def _compute_differential_sharpe_ratio(self, reward, eps=1e-6):
        delta_alpha = reward - self.alpha
        delta_beta = reward**2 - self.beta

        if (self.beta - self.alpha**2) < eps: # prevent blow-up
            dsr = 0
        else:
            dsr = (self.beta*delta_alpha - 0.5*self.alpha*delta_beta) / (self.beta - self.alpha**2)**1.5

        # update
        self.alpha += self.eta * delta_alpha
        self.beta += self.eta * delta_beta

        return dsr


    def _next_observation(self):
        prices = self.df.iloc[self.current_step - self.window_size:self.current_step][['open', 'high', 'low', 'close']].values.flatten()

        self._compute_dual_thrust()
        indicators = [self.buy_line, self.sell_line]

        account = [self.position, self.balance/self.initial_balance]
        return np.concatenate((prices, indicators, account))
    
    def reset(self):
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.position = 0 # 0: no position, 1: long, -1: short
        self.entry_price = 0
        self.buy_line = 0
        self.sell_line = 0

        # first observation (update lines)
        self._compute_dual_thrust()
        return self._next_observation()


    def step(self, action):
            done = False
            if self.current_step >= len(self.df) - 1:
                done = True
            
            if action == 1:
                desired_position = 1
            elif action == 2:
                desired_position = -1
            else:
                desired_position = self.position

            price_open = self.df['open'].iloc[self.current_step]
            
            # Close hold open new
            if desired_position != self.position:
                # close old
                if self.position != 0:
                    old_pnl = (price_open - self.entry_price) * self.position
                    cost = (abs(self.position - desired_position) * self.transaction_fee * price_open 
                            + abs(self.position)*self.slippage)
                    self.balance += old_pnl - cost
                
                # open new
                if desired_position != 0:
                    self.entry_price = price_open
                    cost = (abs(self.position - desired_position) * self.transaction_fee * price_open
                            + abs(desired_position)*self.slippage)
                    self.balance -= cost
                
                self.position = desired_position
            
            # next step
            self.current_step += 1
            if self.current_step < len(self.df):
                self._compute_dual_thrust()
            
            price_close = self.df['close'].iloc[self.current_step - 1]
            step_pnl = (price_close - self.entry_price) * self.position
            
            dsr = self._compute_differential_sharpe_ratio(step_pnl)
            obs = self._next_observation()
            
            if self.balance <= 0:
                done = True
            
            return obs, dsr, done, {}

In [5]:
def dt_policy(env):

    o = env._next_observation()
    n = env.window_size

    buy_line = o[4*n]
    sell_line= o[4*n + 1]

    curr = env.df['open'].iloc[env.current_step-1]

    if curr > buy_line:
        return 1
    elif curr < sell_line:
        return 2
    else:
        return 0
    

def intraday_greedy_actions(env_df, window_size=60):
    """
    Returns an array of 'expert' actions for each step, 
    picking +1 at day's min-open and -1 at day's max-open.
    """
    n_steps = len(env_df)
    day_len = 240
    actions = np.zeros(n_steps, dtype=int)
    
    i = window_size
    while i < (n_steps - 1):
        day_start = i
        day_end = min(i + day_len, n_steps)
        day_slice = env_df.iloc[day_start:day_end]
        day_opens = day_slice['open'].values
        idx_min = day_opens.argmin()
        idx_max = day_opens.argmax()
        
        actions[day_start + idx_min] = 1
        actions[day_start + idx_max] = 2
        i = day_end
    return actions

In [6]:
class Episode:
    def __init__(self):
        self.obs = []
        self.actions = []
        self.rewards = []
        self.next_obs = []
        self.done_flags = []
        self.expert_actions = []  # 'prophetic' (intraday actions)

class RBuffer:
    def __init__(self, max_episodes=1000):
        self.max_episodes = max_episodes
        self.episodes = []
    
    def add_episode(self, ep: Episode):
        if len(self.episodes) >= self.max_episodes:
            self.episodes.pop(0)
        self.episodes.append(ep)
    
    def sample(self, batch_size):
        indices = np.random.randint(0, len(self.episodes), size=batch_size)
        return [self.episodes[i] for i in indices]
    
    def __len__(self):
        return len(self.episodes)

In [7]:
class iRDPGAgent(nn.Module):
    def __init__(self, obs_dim, action_dim=3, hidden_dim=64):
        super().__init__()

        self.actor_gru = nn.GRU(obs_dim, hidden_dim, batch_first=True) # [batch, seq_len, obs_dim]
        self.actor_fc = nn.Linear(hidden_dim, action_dim) # [batch, seq_len, action_dim]

        self.critic_gru = nn.GRU(obs_dim + action_dim, hidden_dim, batch_first=True) # [batch, seq_len, obs_dim + action_dim]
        self.critic_fc = nn.Linear(hidden_dim, action_dim) # [batch, seq_len, action_dim]

    def forward(self, obs, h_actor, h_critic):
        # Actor
        z_actor, h_actor = self.actor_gru(obs, h_actor)
        logits = self.actor_fc(z_actor)
        # Critic
        z_critic, h_critic = self.critic_gru(obs, h_critic)
        q_value = self.critic_fc(z_critic[:, -1])

        return logits, q_value, h_actor, h_critic

In [8]:
def collect_episode(env, agent, noise):

    ep = Episode()
    obs = env.reset()
    done = False
    
    while not done:

        with torch.no_grad():
            obs_t = torch.tensor(obs, dtype=torch.float, device=device).view(1,1,-1)
            logits, Q_vals, _, _ = agent(obs_t)
            # pick a policy action
            probs = torch.softmax(logits[0,0,:], dim=-1)
            if noise:
                dist = torch.distributions.Categorical(probs)
                action = dist.sample().item()
            else:
                action = probs.argmax().item()
        
        next_obs, reward, done, _ = env.step(action)
        
        ep.obs.append(obs)
        ep.actions.append(action)
        ep.rewards.append(reward)
        ep.next_obs.append(next_obs)
        ep.done_flags.append(done)
        
        obs = next_obs
    
    return ep

def collect_demonstrations(df, window_size=60, n_episodes=50):
    demos = []
    env = POMDPTEnv(df, window_size=window_size)
    for _ in range(n_episodes):
        ep = Episode()
        obs = env.reset()
        done = False
        while not done:
            a = dt_policy(env)
            next_obs, rew, done, _ = env.step(a)
            ep.obs.append(obs)
            ep.actions.append(a)
            ep.rewards.append(rew)
            ep.next_obs.append(next_obs)
            ep.done_flags.append(done)
            obs = next_obs
        demos.append(ep)
    return demos

In [9]:
def train_iRDPG(df, window_size=60, hidden_dim=64,
                train_episodes=500, batch_size=8, gamma=0.99,
                tau=0.01):
    
    env = POMDPTEnv(df, window_size=window_size)
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.n
    
    agent = iRDPGAgent(obs_dim, act_dim, hidden_dim).to(device)
    target_agent = iRDPGAgent(obs_dim, act_dim, hidden_dim).to(device)
    target_agent.load_state_dict(agent.state_dict())
    
    # Create a replay buffer
    buffer = RBuffer(max_episodes=1000)
    
    # Collect some demonstration episodes
    demo_episodes = collect_demonstrations(df, window_size=window_size, n_episodes=50)
    for ep in demo_episodes:
        buffer.add_episode(ep)
    
    # Create "prophetic" array for entire df:
    prophecy = intraday_greedy_actions(df, window_size=window_size)
    
    # Attach them to each Episode so we can do Q-filter BC later
    for ep in buffer.episodes:
        ep.expert_actions = []
        # naive mapping: the environment index starts at window_size for step=0
        # so step i -> global index = window_size + i
        idx_global = window_size
        for t in range(len(ep.obs)):
            if idx_global < len(prophecy):
                ep.expert_actions.append(prophecy[idx_global])
            else:
                ep.expert_actions.append(0)
            idx_global += 1
    
    optimizer = nn.optim.Adam(agent.parameters(), lr=1e-3)
    ce = nn.CrossEntropyLoss(reduction='none')
    
    def soft_update(net, net_targ, tau):
        for p, p_targ in zip(net.parameters(), net_targ.parameters()):
            p_targ.data.copy_(tau * p.data + (1.0 - tau) * p_targ.data)
    
    def update_agent(batch_eps):
        """
        Unroll entire episodes on GPU for BPTT.
        """
        total_loss = 0.0
        agent.train()
        optimizer.zero_grad()
        
        for ep in batch_eps:
            obs_seq = torch.tensor(ep.obs, dtype=torch.float, device=device)
            actions_seq = torch.tensor(ep.actions, dtype=torch.long, device=device)
            rewards_seq = torch.tensor(ep.rewards, dtype=torch.float, device=device)
            exp_seq = torch.tensor(ep.expert_actions, dtype=torch.long, device=device)
            
            # shape => [1, T, obs_dim]
            obs_seq = obs_seq.unsqueeze(0) 
            
            logits, Q_vals, _, _ = agent(obs_seq)  # [1,T,act_dim]
            logits = logits[0]  # [T,act_dim]
            Q_vals = Q_vals[0] # [T,act_dim]
            T_len = len(actions_seq)
            
            # Critic loss: basic 1-step TD
            td_errors = []
            for t in range(T_len):
                q_sa = Q_vals[t, actions_seq[t]]
                if t < T_len-1 and not ep.done_flags[t]:
                    # next action from agent policy
                    next_a = logits[t+1].argmax().item()
                    q_next = Q_vals[t+1, next_a].detach()
                    target = rewards_seq[t] + gamma * q_next
                else:
                    target = rewards_seq[t]
                td_errors.append(target - q_sa)
            
            td_errors = torch.stack(td_errors)
            critic_loss = (td_errors**2).mean()
            
            # Actor loss: - Q(s, a_pi)
            pi = torch.softmax(logits, dim=-1)
            Q_expected = (pi * Q_vals).sum(dim=-1)
            actor_loss = - Q_expected.mean()
            
            # Behavior cloning with Q-filter
            a_agent = logits.argmax(dim=-1)
            Q_expert = Q_vals[torch.arange(T_len), exp_seq]
            Q_agent  = Q_vals[torch.arange(T_len), a_agent]
            mask = (Q_expert > Q_agent).float()
            
            bc_loss_t = ce(logits, exp_seq)  # cross-entropy stepwise
            bc_loss   = (bc_loss_t * mask).mean()
            
            ep_loss = critic_loss + actor_loss + bc_loss
            ep_loss.backward(retain_graph=True)
            total_loss += ep_loss.item()
        
        nn.utils.clip_grad_norm_(agent.parameters(), 5.0)
        optimizer.step()
        
        soft_update(agent, target_agent, tau)
        
        return total_loss / len(batch_eps)
    
    # main training loop
    update_trange = trange(train_episodes, desc="Training")
    for ep_i in update_trange:
        new_ep = collect_episode(env, agent=agent, noise=True)
        # attach prophecy to new episode
        new_ep.expert_actions = []
        idx_global = window_size
        for t in range(len(new_ep.obs)):
            if idx_global < len(prophecy):
                new_ep.expert_actions.append(prophecy[idx_global])
            else:
                new_ep.expert_actions.append(0)
            idx_global += 1
        
        buffer.add_episode(new_ep)
        
        if len(buffer) >= batch_size:
            batch_eps = buffer.sample(batch_size)
            loss_val = update_agent(batch_eps)
        else:
            loss_val = 0.0
        update_trange.set_postfix(loss=loss_val)
        
        print(f"Episode {ep_i+1}/{train_episodes}, Loss={loss_val:.4f}")
    
    return agent, target_agent

In [ ]:
t = 10_000

trained, target = train_iRDPG(df=df, window_size=60, train_episodes=t, batch_size=8, gamma=0.99, tau=0.01)

In [ ]:
import numpy as np
import torch

def evaluate_policy(agent, env, n_episodes=5, render=False):
    """
    Runs a given agent's policy for n_episodes in the given environment
    and returns some evaluation metrics.
    
    Returns:
        - average_final_balance
        - average_total_pnl
        - list of final balances for each episode
        - list of step-by-step balances (for optional analysis)
    """
    agent.eval()  # put the network in eval mode
    
    initial_balance = env.initial_balance
    final_balances = []
    step_balances_all = []
    
    for ep_i in range(n_episodes):
        obs = env.reset()
        done = False
        
        # Keep track of the environment's balance at each step
        step_balances = [env.balance]
        
        while not done:
            # In the differential Sharpe environment, the environment's "reward"
            # is not the usual PnL—so we track actual positions/balance ourselves.
            with torch.no_grad():
                # shape = [1,1,obs_dim]
                obs_t = torch.tensor(obs, dtype=torch.float).view(1,1,-1)
                # forward pass through agent
                logits, Q_vals, _, _ = agent(obs_t)
                probs = torch.softmax(logits[0,0,:], dim=-1)
                action = probs.argmax().item()  # greedy action
                
            obs, reward, done, info = env.step(action)
            step_balances.append(env.balance)
            
            if render:
                print(f"Step: {env.current_step}, Action: {action}, Balance: {env.balance:.2f}")
        
        final_balances.append(env.balance)
        step_balances_all.append(step_balances)
    
    final_balances = np.array(final_balances)
    avg_final_balance = final_balances.mean()
    avg_total_pnl = (final_balances - initial_balance).mean()
    
    print(f"Evaluation over {n_episodes} episodes:")
    print(f"  Average Final Balance: {avg_final_balance:.2f}")
    print(f"  Average Total PnL:     {avg_total_pnl:.2f}")
    
    return avg_final_balance, avg_total_pnl, final_balances, step_balances_all